In [ ]:
import sys #guarantees same kernel
!{sys.executable} -m pip install pandas
!{sys.executable} -m pip install requests

#### Quick start Kalshi API  
https://docs.kalshi.com/getting_started/terms/

- **Market:** The tradable binary contract (ticker, bid/ask, volume, orderbook, candles, etc.).
- **Event:** The specific real-world occurrence (e.g., one game date/matchup); contains one or more markets.
- **Series:** A collection of related events that share the same rules/structure (same contract type).
  - **Example:** “NBA game winner” as a repeating template for every game.

- **Kalshi data has the following hierarchy (3 levels):**
  - **Series:** `KXNBAGAME` (series ticker in the API; case-sensitive)
    - **Event:** `kxnbagame-26feb24nykcle`
      - **Markets:** one or more market tickers inside the event (often one per team for “winner” style events)

- **Common market fields you’ll see:** 
  - `yes_bid`, `yes_ask`, `last_price`, `volume`, `open_interest`, `orderbook`, `candlesticks`, `trades`

Let's start by fetching information about the KXNBAGAME series. We'll use the Get Series endpoint.

In [ ]:
import requests

# Get series information for KXNBAGAME
url = "https://api.elections.kalshi.com/trade-api/v2/series/KXNBAGAME"
response = requests.get(url)
series_data = response.json()

print(f"Series Title: {series_data['series']['title']}")
print(f"Frequency: {series_data['series']['frequency']}")
print(f"Category: {series_data['series']['category']}")

Series Title: Professional Basketball Game
Frequency: custom
Category: Sports


Now that we have the series information, lets get the markets for this series

In [ ]:
# Get all open markets for the KXNBAGAME series
markets_url = f"https://api.elections.kalshi.com/trade-api/v2/markets?series_ticker=KXNBAGAME&status=open"
markets_response = requests.get(markets_url)
markets_data = markets_response.json()

print(type(markets_data))
print(list(markets_data["markets"][0].keys()))

In [ ]:
# Display the first market in the series as a df
import pandas as pd

df = pd.json_normalize(markets_data["markets"][0])
df.head()
#df.columns.tolist() # list column names

,can_close_early,close_time,created_time,early_close_condition,event_ticker,expected_expiration_time,expiration_time,expiration_value,fractional_trading_enabled,last_price,...,volume,volume_24h,volume_24h_fp,volume_fp,yes_ask,yes_ask_dollars,yes_bid,yes_bid_dollars,yes_sub_title,custom_strike.basketball_team
0,True,2026-03-12T03:00:00Z,2026-02-23T17:02:05.873203Z,This market will close and expire after a winn...,KXNBAGAME-26FEB25BOSDEN,2026-02-26T06:00:00Z,2026-03-12T03:00:00Z,,False,0,...,0,0,0.00,0.00,63,0.6300,51,0.5100,Denver,ba2cb6be-9827-46a8-8e6f-d8661307ba77


The fields describe: what the contract is, when it trades/closes, and the current order book / trading stats.

##### In any market with an order book:
- **Bid** = the best price someone is currently offering to buy at.
- **Ask** = the best price someone is currently offering to sell at.

- So for **YES** in your output:
    - `yes_bid`: 46 → highest buyer is willing to pay 46¢ for YES ( $0.46 )
    - `yes_ask`: 63 → lowest seller is asking 63¢ for YES ( $0.63 )
    - `no_bid`, no_ask: best buy / best sell for NO
- For **NO**:
    - `no_bid`: 37 ( $0.37 )
    - `no_ask`: 54 ( $0.54 )

##### Activity / liquidity
- `volume`, `volume_24h`: number of contracts traded total / last 24h
- `open_interest`: how many contracts are currently open (held) and not settled


In [ ]:
# quick look at all the markets in this series
print(f"\nActive markets in KXNBAGAME series:")
for market in markets_data['markets']:
    print(f"- {market['ticker']}: {market['title']}")
    print(f"  Event: {market['event_ticker']}")
    print(f"  Yes Bid: {market['yes_bid']}¢ | Volume: {market['volume']}")
    print(f"  Yes Ask: {market['yes_ask']}¢ | Volume: {market['volume']}")
    print()

In [ ]:
# all of the markets 
markets_data[markets_data['markets']] 

In [ ]:
# Lets take a closer look at the knicks game 
# Get details for event_ticker: KXNBAGAME-26FEB24NYKCLE 
event = "KXNBAGAME-26FEB24NYKCLE"

event_markets = [
    m for m in markets_data["markets"]
    if m.get("event_ticker") == event
]
event_markets  # list of all markets in that event

##### Closer look at the output 

You’ve got two separate binary markets inside the same event (`KXNBAGAME-26FEB24NYKCLE`). Each market answers the same question (“Winner?”) but with a different team as “YES”.

What are the different markets here?
1) `KXNBAGAME-26FEB24NYKCLE-NYK`
- YES means: New York wins (see rules_primary + yes_sub_title: 'New York')
    - Prices:
        - `yes_bid`/`yes_ask`: 40–41¢ (buyers at 40, sellers at 41)
        - `no_bid`/`no_ask`: 59–60¢ (NO = “New York does not win”)
This is basically the “Bet on NYK to win” contract.

2) `KXNBAGAME-26FEB24NYKCLE-CLE`
- YES means: Cleveland wins (yes_sub_title: 'Cleveland')
    - Prices:
        - `yes_bid`/`yes_ask`: 60–62¢
        - `no_bid`/`no_ask`: 38–40¢
This is the “Bet on CLE to win” contract.

Important intuition: These two markets are “opposites” in outcome (if one team wins, the other doesn’t), but they’re still different tickers with separate order books, so their prices/spreads/volume/open interest can differ.

In [40]:
# Get details for event_ticker: KXNBAGAME-26FEB24NYKCLE 

if markets_data['markets']:
    # Let's get details for the first market's event
    #event_ticker = markets_data['markets'][1]['event_ticker']
    event_ticker = 'KXNBAGAME-26FEB24NYKCLE'
    event_url = f"https://api.elections.kalshi.com/trade-api/v2/events/{event_ticker}"
    event_response = requests.get(event_url)
    event_data = event_response.json()

    print(f"Event Details:")
    print(f"Title: {event_data['event']['title']}")
    print(f"Category: {event_data['event']['category']}")

Event Details:
Title: New York at Cleveland
Category: Sports


##### Get Orderbook Data
Lets fetch the orderbook for a specific market to see the current bids and asks using the Get Market orderbook endpoint.

In [49]:
# Get orderbook for a specific market 'ticker': 'KXNBAGAME-26FEB24NYKCLE-NYK',
# Replace with an actual market ticker from the markets list
if not markets_data['markets']:
    raise ValueError("No open markets found. Try removing status=open or choose another series.")

# market_ticker = markets_data['markets'][0]['ticker']
market_ticker = 'KXNBAGAME-26FEB24NYKCLE-NYK'
orderbook_url = f"https://api.elections.kalshi.com/trade-api/v2/markets/{market_ticker}/orderbook"

orderbook_response = requests.get(orderbook_url)
orderbook_data = orderbook_response.json()

print(f"\nOrderbook for {market_ticker}:")
print("YES BIDS:")
for bid in orderbook_data['orderbook']['yes'][:5]:  # Show top 5
    print(f"  Price: {bid[0]}¢, Quantity: {bid[1]}")

print("\nNO BIDS:")
for bid in orderbook_data['orderbook']['no'][:5]:  # Show top 5
    print(f"  Price: {bid[0]}¢, Quantity: {bid[1]}")


Orderbook for KXNBAGAME-26FEB24NYKCLE-NYK:
YES BIDS:
  Price: 1¢, Quantity: 1437368
  Price: 2¢, Quantity: 4900
  Price: 3¢, Quantity: 172214
  Price: 5¢, Quantity: 66609
  Price: 7¢, Quantity: 38456

NO BIDS:
  Price: 1¢, Quantity: 1598477
  Price: 2¢, Quantity: 4900
  Price: 3¢, Quantity: 181609
  Price: 5¢, Quantity: 72241
  Price: 7¢, Quantity: 45006


Kalshi’s /orderbook response is resting BID orders only (i.e., “I’m willing to buy at this price”). It returns bids for YES and bids for NO; it does not return asks.

yes: [[1, 1437368], ...] means
there are buy orders to buy YES at 1¢ totaling 1,437,368 contracts.

no: [[1, 1598477], ...] means
there are buy orders to buy NO at 1¢ totaling 1,598,477 contracts.

That’s common in order books: people (or market makers) leave “lottery ticket” bids far away from the current price. They only fill if someone is willing to sell that low.

##### If you want to see what price people actually traded at

In [ ]:
ob = orderbook_data["orderbook"]

best_yes_bid = max(p for p, q in ob["yes"]) # generator expression
best_no_bid  = max(p for p, q in ob["no"]) #

implied_yes_ask = 100 - best_no_bid

print("Best YES bid:", best_yes_bid, "¢")
print("Implied YES ask:", implied_yes_ask, "¢")
print("Spread:", implied_yes_ask - best_yes_bid, "¢")

Best YES bid: 40 ¢
Implied YES ask: 41 ¢
Spread: 1 ¢
